## TASK 5

In [5]:
source("./Imports.R")

## Description
Select 7 columns: Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location from given PostsBestComments table:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;OwnerUserId, Title, CommentCount, ViewCount, CommentsTotalScore from given CmtToScr table:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;PostId, CommentsTotalScore from Comments  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;CommentsTotalScore is sum of Score  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Grouped by PostId  
Joins PostsBestComments table and Users table by user id  
Orders by CommentsTotalScore (descending)  
Takes 10 first results  
### Effectively returns top 10 Users with highest CommentsTotalScore


In [6]:
sql_5 <- function(Posts, Comments, Users){
  sqldf("SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
        FROM (
            SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,CmtTotScr.CommentsTotalScore
            FROM (
                SELECT PostId, SUM(Score) AS CommentsTotalScore
                FROM Comments
                GROUP BY PostId
                ) AS CmtTotScr
            JOIN Posts ON Posts.Id = CmtTotScr.PostId
            WHERE Posts.PostTypeId=1
        ) AS PostsBestComments
    JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
    ORDER BY CommentsTotalScore DESC
    LIMIT 10
  ")
}

In [7]:
base_5 <- function(Posts,Comments,Users) {
  cmtTotScr <- aggregate(Comments$Score ~ Comments$PostId, data = Comments, sum)
  names(cmtTotScr) <- c("PostId", "CommentsTotalScore")
  postsBestCmt <- merge(Posts[Posts$PostTypeId == 1,], cmtTotScr, by.x = "Id", by.y = "PostId")
  result <- merge(postsBestCmt, Users, by.x = "OwnerUserId", by.y = "Id")
    
  result <- result[order(-result$CommentsTotalScore), ]
  result <- result[1:10, c("Title", "CommentCount", "ViewCount", "CommentsTotalScore", 
                              "DisplayName", "Reputation", "Location")]
  return(result)
}

In [8]:
dplyr_5 <- function(Posts, Comments, Users){
  postsBestComments <- Comments %>%
    group_by(PostId) %>%
    summarise(CommentsTotalScore = sum(Score)) %>%
    inner_join(Posts %>% 
                 filter(PostTypeId == 1),by = c("PostId" = "Id"))
  Users %>% 
    inner_join(postsBestComments, by = c("Id" = "OwnerUserId")) %>%
    arrange(desc(CommentsTotalScore)) %>%
    head(10) %>%
    select(Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location)
    
}

In [9]:
table_5 <- function(Posts, Comments, Users){
  setDT(Posts)
  setDT(Users)    
  setDT(Comments)
  
  cmtTotScr <- Comments[, .(CommentsTotalScore = sum(Score)), by = PostId]
  PostsBestComments <- merge(Posts[PostTypeId == 1], cmtTotScr, by.x = "Id", by.y = "PostId")
  merge(PostsBestComments, Users, by.x = "OwnerUserId", by.y = "Id")[order(-CommentsTotalScore)][1:10][,.(Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location)]
  
}

In [10]:
comp(
    sql_5(Posts,Comments,Users),
    base_5(Posts,Comments,Users),
    dplyr_5(Posts,Comments,Users),
    table_5(Posts,Comments,Users)
)

[1] "COMPARING GIVEN OUTPUTS TO SQL ONE: "
[1] "BASE: TRUE"
[1] "DPLYR: TRUE"
[1] "TABLE: TRUE"


In [11]:
table_5(Posts,Comments,Users)

Title,CommentCount,ViewCount,CommentsTotalScore,DisplayName,Reputation,Location
<chr>,<int>,<int>,<int>,<chr>,<int>,<chr>
"Boss is asking for passport, but it has a stamp in it I don't want him to see. What to do?",24,60532,623,Megha,1639,
"How to intentionally get denied entry to the US, without getting into trouble?",37,71829,601,user11743,1947,
Can I wear a bulletproof vest while traveling on British Airways from Heathrow?,26,22958,401,Ulkoma,8413,"London, United Kingdom"
"OK we're all adults here, so really, how on earth should I use a squat toilet?",29,107591,380,hippietrail,74614,"Sunshine Coast QLD, Australia"
Being separated from one's young children on a flight,36,13974,358,Emma-louise,611,
Immigration officer that stopped me at the airport is texting me. What do I do?,24,21808,345,DiegoJ,1243,Buenos Aires
How to avoid toddlers on a long-distance plane flight?,18,28968,342,Silver Dragon,933,"San Francisco, United States"
How to book a flight if my passport doesn't state my gender?,23,17614,329,kukis,894,"Sztokholm, Szwecja"
Why don't airlines have backup planes just in case of an emergency?,26,18365,328,clickbait,532,
